In [13]:
# @zhangjoy
# @主题挖掘
import csv
import time
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import tushare as ts
import jieba
import re
from sklearn.feature_extraction.text import CountVectorizer # 词频统计
from sklearn.decomposition import LatentDirichletAllocation # LDA
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer # tf-idf统计
from sklearn.cluster import KMeans # k均值聚类

In [14]:
# 评论文本：comment
filepath = "D:\\Program Files\\Git\\\git_work\\weibo-text-project\\data\\cjpl_20180614_20180616.csv"   # 评论内容的存储路径
with open(filepath, encoding='gb18030') as f:
    reader = csv.reader(f)
    comment = [row[2] for row in reader]
comment.pop(0) # '评论内容'
print(len(comment)) # 6710

108


In [15]:
comment[101]

'来源：中国证券网 编辑：东方财富网记者14日从财政部获悉，5月份，全国一般公共预算收入17631亿元，同比增长9.7%，全国一般公共预算支出17003亿元，同比增长0.5%。1-5月累计，全国一般公共预算收入86650亿元，同比增长12.2%。其中，证券交易印花税602亿元，同比增长12.7%。全国一般公共预算支出82695亿元，同比增长8.1%，为年初预算的39.4%，比去年同期进度加快0.2个百分点。数据明细如下一、全国一般公共预算收支情况(一)一般公共预算收入情况。5月份，全国一般公共预算收入17631亿元，同比增长9.7%。其中，中央一般公共预算收入8650亿元，同比增长8.8%；地方一般公共预算本级收入8981亿元，同比增长10.6%。全国一般公共预算收入中的税收收入15912亿元，同比增长12.8%；非税收入1719亿元，同比下降12.8%。1-5月累计，全国一般公共预算收入86650亿元，同比增长12.2%。其中，中央一般公共预算收入41880亿元，同比增长15.3%；地方一般公共预算本级收入44770亿元，同比增长9.5%。全国一般公共预算收入中的税收收入76810亿元，同比增长15.8%；非税收入9840亿元，同比下降9.5%。1-5月累计，主要收入项目情况如下：1、国内增值税28392亿元，同比增长19%。主要是工业生产增速回升，服务业景气持续提升，价格上涨等因素带动。2、国内消费税6031亿元，同比增长21.1%。主要是卷烟、成品油产销量增长带动。3、企业所得税19319亿元，同比增长10.9%。主要是企业利润增长带动。4、个人所得税6891亿元，同比增长20.6%。主要是城镇居民收入增加以及纳税人数增长带动。5、进口货物增值税、消费税7236亿元，同比增长14.8%；关税1253亿元，同比增长5.2%。主要是一般贸易进口增长带动。6、出口退税6161亿元，同比增长10.8%。7、城市维护建设税2177亿元，同比增长17.5%。8、车辆购置税1561亿元，同比增长21.9%。9、印花税1150亿元，同比增长12.5%。其中，证券交易印花税602亿元，同比增长12.7%。10、资源税697亿元，同比增长23%。11、环境保护税46亿元(今年新开征)。12、土地和房地产相关税收中，契税2402亿元，同比增长18.1%；土地增值税2490亿元，同比

In [16]:
# comment可视化分析
l_lst = [len(c) for c in comment]

In [17]:
max(l_lst)

7420

In [18]:
min(l_lst)

91

In [19]:
sum(l_lst) / len(l_lst)

1638.2037037037037

In [151]:
t_lst = [466,511,348,195,689,584,451,355,315,128,123,195,202,178,52,50,51,32,48,43,56,303,302,131,67,272,299,263]

In [152]:
sum(t_lst)

6709

In [153]:
# 得到所有股票的代码和中文名字
stock_name = ts.get_stock_basics()['name'] # 股票名称
stock_code = stock_name.index.tolist() # 股票代码
# 获得所有的概念列表、行业列表、地域列表、公司名称等
concept = ts.get_concept_classified()['c_name'].drop_duplicates().values.tolist() # 概念列表 163
industry = ts.get_industry_classified('sw')['c_name'].drop_duplicates().values.tolist() # 申万行业列表 129 > 110
place = ts.get_area_classified()['area'].drop_duplicates().values.tolist() # 获取地域列表 32 省份

broker = ts.broker_tops()["broker"] # 获取营业部信息：1062
pattern = re.compile(u'.*有限.*?公司') # 最小匹配[*?]到'有限 公司'
company = [" ".join(pattern.findall(s)) for s in broker] # list
company = pd.Series(company).drop_duplicates().values.tolist() # 证券公司：94

[Getting data:]############################

In [6]:
# 将上述信息作为新词录入词典
for d in stock_name:
    jieba.add_word(d)
for d in stock_code:
    jieba.add_word(d)
for d in concept:
    jieba.add_word(d)
for d in industry:
    jieba.add_word(d)
for d in place:
    jieba.add_word(d)    
for d in company:
    jieba.add_word(d)

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/xv/52pzd4c57hb49kf41gxp713w0000gn/T/jieba.cache
Loading model cost 0.927 seconds.
Prefix dict has been built succesfully.


In [20]:
# 启用自定义词典
jieba.load_userdict("D:\\Program Files\\Git\\git_work\\weibo-text-project\\userdict.txt")

In [21]:
# 停用词列表
stop =[line.strip() for line in open('D:\\Program Files\\Git\\git_work\\weibo-text-project\\mystopwords.txt', encoding='utf-8').readlines()]

In [22]:
# 匹配中文
pattern = re.compile(u'[\u4e00-\u9fa5]{2,20}')

In [23]:
text = [] # 清洗后文本
t1 = time.time()
for sentence in comment:
    chinese = pattern.findall(sentence) # 匹配中文：先分句
    seg = [jieba.cut(c) for c in chinese] # 双重列表：句上分词
    c = []
    for ss in seg:
        for s in ss:
            if s not in stop: # 过滤停用词
                c.append(s)
    text.append(" ".join(c))
t2 = time.time()
print("preprocessing time using %.5f s" %(t2-t1)) # 预处理时间 111.45185 s

preprocessing time using 3.70456 s


In [24]:
text[101]

'中国证券网 东方财富网 记者 日 财政部 获悉 全国 公共 预算收入 同比 增长 全国 公共 预算 支出 同比 增长 月 全国 公共 预算收入 同比 增长 证券交易印花税 同比 增长 全国 公共 预算 支出 同比 增长 年初 预算 去年同期 进度 加快 百分点 数据 明细 全国 公共 预算 收支 公共 预算收入 全国 公共 预算收入 同比 增长 中央 公共 预算收入 同比 增长 公共 预算 本级 收入 同比 增长 全国 公共 预算收入 中 税收收入 同比 增长 非税收入 同比 下降 月 全国 公共 预算收入 同比 增长 中央 公共 预算收入 同比 增长 公共 预算 本级 收入 同比 增长 全国 公共 预算收入 中 税收收入 同比 增长 非税收入 同比 下降 月 收入 项目 国内 增值税 同比 增长 工业生产 增速 回升 服务业 景气 持续提升 价格上涨 因素 带动 国内 消费税 同比 增长 卷烟 成品油 产销量 增长 带动 企业 所得税 同比 增长 企业 利润 增长 带动 个人所得税 同比 增长 城镇 居民收入 增加 纳税 人数 增长 带 进口货 物 增值税 消费税 同比 增长 关税 同比 增长 贸易 进口 增长 带动 出口退税 同比 增长 城市 维护 建设 税 同比 增长 车辆 购置税 同比 增长 印花税 同比 增长 证券交易印花税 同比 增长 资源税 同比 增长 环境保护 税 新 开征 土地 房地产 税收 中 契税 同比 增长 土地 增值税 同比 增长 房产税 同比 增长 耕地 占用 税 同比 下降 城镇 土地 使用税 同比 增长 车船税 船舶 吨 税 烟叶 税 各项 税收收入 合计 同比 增长 公共 预算 支出 全国 公共 预算 支出 同比 增长 中央 公共 预算 本级 支出 同比 增长 公共 预算 支出 同比 下降 支 出错 月 去年同期 基数 高等 因素 月 全国 公共 预算 支出 同比 增长 年初 预算 去年同期 进度 加快 百分点 中央 公共 预算 本级 支出 同比 增长 公共 预算 支出 同比 增长 月 支出 科目 教育 支出 同比 增长 科学技术 支出 同比 增长 文化 体育 传媒 支出 同比 增长 社会保障 就业 支出 同比 增长 医疗卫生 计划生育 支出 同比 增长 节能环保 支出 同比 增长 城乡 社区 支出 同比 增长 农林 水 支出 同

In [25]:
# 使第101条评论完美分词
jieba.load_userdict("D:\\Program Files\\Git\\git_work\\weibo-text-project\\userdict_weibo.txt")
stop =[line.strip() for line in open('D:\\Program Files\\Git\\git_work\\weibo-text-project\\mystopwords.txt', encoding='utf-8').readlines()]
text_101 = "" # 清洗后文本
comment_101 = comment[101]
chinese = pattern.findall(comment_101) # 匹配中文：先分句
seg = [jieba.cut(c) for c in chinese] # 双重列表：句上分词
c = []
for ss in seg:
    for s in ss:
        if s not in stop: # 过滤停用词
            c.append(s)
text_101 = " ".join(c)
print(text_101)

中国证券网 东方财富网 记者 日 财政部 获悉 全国 公共 预算收入 同比 增长 全国 公共 预算 支出 同比 增长 月 全国 公共 预算收入 同比 增长 证券交易印花税 同比 增长 全国 公共 预算 支出 同比 增长 年初 预算 去年同期 进度 加快 百分点 数据 明细 全国 公共 预算 收支 公共 预算收入 全国 公共 预算收入 同比 增长 中央 公共 预算收入 同比 增长 公共 预算 本级 收入 同比 增长 全国 公共 预算收入 中 税收收入 同比 增长 非税收入 同比 下降 月 全国 公共 预算收入 同比 增长 中央 公共 预算收入 同比 增长 公共 预算 本级 收入 同比 增长 全国 公共 预算收入 中 税收收入 同比 增长 非税收入 同比 下降 月 收入 项目 国内 增值税 同比 增长 工业生产 增速 回升 服务业 景气 持续提升 价格上涨 因素 带动 国内 消费税 同比 增长 卷烟 成品油 产销量 增长 带动 企业 所得税 同比 增长 企业 利润 增长 带动 个人所得税 同比 增长 城镇 居民收入 增加 纳税 人数 增长 带 进口货 物 增值税 消费税 同比 增长 关税 同比 增长 贸易 进口 增长 带动 出口退税 同比 增长 城市 维护 建设 税 同比 增长 车辆 购置税 同比 增长 印花税 同比 增长 证券交易印花税 同比 增长 资源税 同比 增长 环境保护 税 新 开征 土地 房地产 税收 中 契税 同比 增长 土地 增值税 同比 增长 房产税 同比 增长 耕地 占用 税 同比 下降 城镇 土地 使用税 同比 增长 车船税 船舶 吨 税 烟叶 税 各项 税收收入 合计 同比 增长 公共 预算 支出 全国 公共 预算 支出 同比 增长 中央 公共 预算 本级 支出 同比 增长 公共 预算 支出 同比 下降 支 出错 月 去年同期 基数 高等 因素 月 全国 公共 预算 支出 同比 增长 年初 预算 去年同期 进度 加快 百分点 中央 公共 预算 本级 支出 同比 增长 公共 预算 支出 同比 增长 月 支出 科目 教育 支出 同比 增长 科学技术 支出 同比 增长 文化 体育 传媒 支出 同比 增长 社会保障 就业 支出 同比 增长 医疗卫生 计划生育 支出 同比 增长 节能环保 支出 同比 增长 城乡 社区 支出 同比 增长 农林 水 支出 同比

In [26]:
corpus = text # 清洗后文本作为语料

In [27]:
# LDA
# 特征处理: 词频做文档特征
# 构建 Document-word matrix
count_vectorizer = CountVectorizer(stop_words=stop) # 停用词, 特征选择：min_df = 6
count = count_vectorizer.fit_transform(corpus) # 文档-词频矩阵：<class 'scipy.sparse.csr.csr_matrix'>
word_names = count_vectorizer.get_feature_names() # wordlist 长度：84888

In [28]:
countpie_vectorizer = CountVectorizer(min_df=50,stop_words=stop) # 过滤词出现的文档数 < 50 的词 ：特征提取
countpie = countpie_vectorizer.fit_transform(corpus) # 文档-词频矩阵：<class 'scipy.sparse.csr.csr_matrix'>
print(countpie.shape)

(108, 3)


In [29]:
c_lst = [count.sum(axis=0)[0,i] for i in range(len(word_names))] # 每个词的词频列表 耗时：5m

In [30]:
less_6 = sum([Counter(c_lst)[i] for i in range(6)]) # 词频 < 6 的个数

In [31]:
# 特征选择：基于文档频率，返回文档频率排名前 K 的词语
def word_count_select(df,K):
    df_mean = df.mean() # 求每列的均值/和等
    # 基于 文档频率排序 选择特征，一定返回K个词语
    WordTF_series = df_mean.sort_values(ascending=False) # 降序排序
    WordTF_topK = WordTF_series[:K] # 选择前 K 的特征
    word_feature = WordTF_topK.index # word_name [pandas.core.indexes.base.Index]
    feature_list = list(WordTF_topK.index) # list(word_name)
    return word_feature,feature_list
# 词频小于 6 的个数统计
def count_less_n(lst,N):
    lessN = sum([Counter(lst)[i] for i in range(N)]) # 词频 < N 的个数
    return lessN

In [32]:
# 把文档-词语矩阵转为数据框格式
count_ndarray = count.toarray()
count_df = DataFrame(count_ndarray, columns=word_names)
k = len(word_names) - count_less_n(c_lst,6) # 词频 > 6 的个数

wordstay_names,wordstay_names_list = word_count_select(count_df,k)
count_df2 = count_df[wordstay_names]
countstay = count_df2.as_matrix(columns=wordstay_names_list) # matrix

In [33]:
# count # 6709x84884 sparse matrix numpy.int64
# 去掉低频词：在主题模型中，数据预处理需要去掉低频词，大多数阈值设定为5或6,此处设定为6
# matrix是numpy中的特殊类型，作为array的子类出现，所以继承了array的所有特性并且有自己的特殊的地方，专门用来处理线性代数操作

In [35]:
# 词频数分布情况
f = open('D:\\Program Files\\Git\\git_work\\weibo-text-project\\count_1.txt','w')
f.write(str([Counter(c_lst).most_common(100)[i][0] for i in range(100)]))
f.close()
f = open('D:\\Program Files\\Git\\git_work\\weibo-text-project\\count_2.txt','w')
f.write(str([Counter(c_lst).most_common(100)[i][1] for i in range(100)]))
f.close()

In [36]:
# 主题挖掘建模：LDA算法
# 训练模型
k = 50 # 选择主题数
lda_50 = LatentDirichletAllocation(n_components=k, max_iter=50, learning_method='batch')
lda_50.fit(countstay)
# 6709文档，单词数：25434，分50个主题 耗时：< 10m

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=50, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [37]:
# 训练模型
k = 30 # 选择主题数
lda_30 = LatentDirichletAllocation(n_components=k, max_iter=50, learning_method='batch')
lda_30.fit(countstay)
# 6709文档，单词数：25434，分30个主题 耗时：12:54-13:01 7m

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=30, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [38]:
# 训练模型
k = 10 # 选择主题数
lda_10 = LatentDirichletAllocation(n_components=k, max_iter=50, learning_method='batch')
lda_10.fit(countstay)
# 6709文档，单词数：25434，分10个主题 耗时：13:02-13:08 6m

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [39]:
# 结果展示
# Topic Words 结果
# topicword_matrix = lda.components_ # numpy.ndarray
# print(topicword_matrix[0,1]) # topicword_matrix[i,j]: word j was assigned to topic i
# print(topicword_matrix.shape)
# topicword_matrix / topicword_matrix.sum(axis=1)[:, np.newaxis] # normalization

# Topic（Top）Words结果
def print_top_words(model, feature_names, n_top_words):
    print("# 打印每个主题下权重较高的词")
    topicword_matrix = model.components_ # Topic Words 分布结果 numpy.ndarray
    for topic_idx, topic in enumerate(topicword_matrix):
        print("Topic # %d: " % topic_idx)
        print(''.join([feature_names[i] + ' ' + str(round(topic[i], 2)) + '| ' for i in topic.argsort()[:-n_top_words - 1:-1]]))
        # print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

def print_top_words_name(model, feature_names, n_top_words):
    print("# 打印每个主题下权重较高的词")
    topicword_matrix = model.components_
    for topic_idx, topic in enumerate(topicword_matrix):
#         print("Topic # %d:" % topic_idx)
        print("Topic %d:" % topic_idx, " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))

In [40]:
# Doc Topics 结果
# 通过fit_transform函数,可以得到文档主题分布,Doc_Topic distribution matrix
# doctopic_matrix = lda.fit_transform(dw_countvector) # numpy.ndarray
# print(doctopic_matrix)
# print(doctopic_matrix.shape)

# Doc（Top）Topic结果
def print_top_topics(model, feature_names, n_top_topics):
#     fo = open("/Users/zhangjoy/Documents/vscode/py_project/lda/data/doc_top_topics.txt", "rw+")
    print("# 打印每个文档下权重较高的主题")
    doctopic_matrix = model.fit_transform(countstay)
    for doc_idx, doc in enumerate(doctopic_matrix):
        print("Doc # %d:" % doc_idx)
        print(''.join([str(feature_names[i]) + ' ' + str(round(doc[i], 2)) + '| ' for i in doc.argsort()[:-n_top_topics - 1:-1]]))

In [41]:
print_top_words_name(lda_50, wordstay_names_list, 10)

# 打印每个主题下权重较高的词
Topic 0: 体彩 世界杯 销售 竞彩 彩票 涨停 期间 销量 行业 业务
Topic 1: 尔康制药 深交所 会计师事务所 年报 问询 回函 收入 天健 财务总监 优势
Topic 2: 百分点 黄金 市值 小微 全球 论坛 上海 回落 涨停 增长
Topic 3: 总市值 退市昆机 数据 交易日 东方财富 买入 营业部 最低 股价 成交
Topic 4: 市场 板块 反弹 指数 机会 震荡 加息 概念 消费 股指
Topic 5: 加息 美国 美元 收涨 预期 美联储 美股 周四 前值 市场
Topic 6: 推荐 标的 建议关注 板块 全文 持续 行业 重点 领域 受益
Topic 7: 公告 股份 项目 万股 增持 总股本 股权 交易 计划 晚间
Topic 8: 企业 独角兽 调研 投资者 接待 上市公司 定价 次数 机构 市场
Topic 9: 加息 千山药机 利率 实际 央行 美联储 香港 调查 股市 资金
Topic 10: 发展 核心技术 突破 技术 领域 创新 企业 网络 技术创新 强国
Topic 11: 同比 增长 公共 支出 全国 预算收入 预算 下降 政府 收入
Topic 12: 交易 排污权 企业 周四 二级市场 该股 有偿 使用 排放 产品
Topic 13: 股票 平仓 质押 闪崩 信托 市场 股东 风险 公告 员工持股计划
Topic 14: 国内 升级 黄金 红利 报道 政府 分享 销售 销售额 研究院
Topic 15: 基金 独角兽 销售 销量 记者 银行 人士 招行 产品 招商
Topic 16: 跌停 市场 净流入 质押 持续 股通 大众 集团 股票 德国
Topic 17: 央行 预期 改革 当日 同比 制度 企业 监管 全国 增长
Topic 18: 欧洲央行 欧元 预期 通胀 东方财富网 利率 决定 欧洲 将会 预计
Topic 19: 凯迪 债务 企业 会议 人士 集团 世纪 报道 经济 凯迪生态
Topic 20: 习近平 发展 中方 世界 半岛 政治 解决 合作 双方 积极
Topic 21: 小米 发行 生态 企业 披露 优先股 估值 港股 比例 证监会
Topic 22: 结构调整 指数 央企 基金 上海 国企改革 企业 股票 国企 中证
Topic 23: 团队 信

In [42]:
print_top_words_name(lda_30, wordstay_names_list, 10)

# 打印每个主题下权重较高的词
Topic 0: 千山药机 披露 年报 证监会 调查 经营 刘祥华 违规 集团 意见
Topic 1: 市场 房地产 库存 住房 用户 记者 城市 基石 万平方米 方式
Topic 2: 调研 总市值 投资者 日均 接待 上市公司 退市昆机 次数 数据 亿帆医药
Topic 3: 加息 预期 美国 指数 市场 美联储 利率 欧洲央行 调整 收涨
Topic 4: 公告 股份 券商 评级 买入 增持 万股 总股本 项目 股权
Topic 5: 企业 业务 投资 产业 领域 设计 集成电路 上市公司 销售 电子
Topic 6: 仲裁 全景网 沟通 集体 请求 创造 介绍 年度 上市公司 价值
Topic 7: 改革 住宅 制度 租赁 伟民 用地 加快 允许 建设 城市
Topic 8: 基金 独角兽 销售 规模 销量 基金公司 配售 战略 记者 渠道
Topic 9: 企业 提出 增长 行业 持续 火电 穿透 股数 同比 集团
Topic 10: 指数 结构调整 央企 下半年 基金 策略 五大 兴业证券 股票 创新
Topic 11: 工业富联 总市值 上市 估值 市盈率 毛利率 涨停板 市值 企业 成交额
Topic 12: 交易 排污权 有偿 使用 排放 年度 企业 深深宝 污染物 开展
Topic 13: 人人乐 生鲜 商品 基地 销售 战略 达到 消费 模式 爆款
Topic 14: 技术 集团 市场 价格 服务 经济 升级 大众 中国 增长
Topic 15: 独立 网络 基站 时代 运营商 全球 我国 应用 需求 系统
Topic 16: 加息 央行 利率 实际 股票 平仓 美联储 质押 股市 员工持股计划
Topic 17: 小米 发行 生态 企业 披露 优先股 比例 港股 估值 证监会
Topic 18: 闪崩 跌停 质押 股票 信托 市场 股价 持续 资金 控股股东
Topic 19: 申请 小洋 开发 上海 合作 中心 北侧 加快 建设 审核
Topic 20: 净流出 净流入 机构资金 机构 交易日 检查 成交 金额 行业 证监会
Topic 21: 数码 东蓝 飞利信 梅安森 收入 业绩 项目 确认 智慧 补偿
Topic 22: 市场 反弹 机会 震荡 板块 指数 概念 股指 行业 下跌
Topic 23: 独

In [43]:
print_top_words_name(lda_10, wordstay_names_list, 10)

# 打印每个主题下权重较高的词
Topic 0: 股票 质押 市场 闪崩 结构调整 平仓 指数 跌停 信托 康得新
Topic 1: 基金 销售 预期 规模 美元 美国 独角兽 市场 加息 预计
Topic 2: 数码 东蓝 飞利信 梅安森 券商 评级 收入 买入 业绩 项目
Topic 3: 公告 股份 项目 千山药机 重大 深交所 上市公司 披露 集团 东方财富网
Topic 4: 加息 利率 实际 资金 行业 上市公司 美联储 调研 央行 投资者
Topic 5: 同比 增长 工业富联 企业 全国 独角兽 净利润 核心技术 公共 支出
Topic 6: 小米 发行 企业 生态 披露 证监会 比例 估值 优先股 港股
Topic 7: 市场 板块 指数 行业 反弹 银行 机会 资金 净流入 震荡
Topic 8: 发展 企业 新能源 汽车 建设 负责 技术 领域 产业 创新
Topic 9: 市场 改革 制度 机构 房地产 总市值 住房 央行 房价 企业


In [44]:
# 训练模型
k = 20 # 选择主题数
lda_20 = LatentDirichletAllocation(n_components=k, max_iter=50, learning_method='batch')
lda_20.fit(countstay)
# 6709文档，单词数：25434，分20个主题 耗时：17:28-17:35 7m

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=20, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [45]:
print_top_words_name(lda_20, wordstay_names_list, 30)

# 打印每个主题下权重较高的词
Topic 0: 交易 消费 电子 高送转 排污权 资金 净利润 企业 增幅 有偿 大族激光 使用 排放 春兴精工 行业 板块 北上 同比 污染物 除权 除息 最高 激光 业绩 持股 指数 产生 开展 达到 增加
Topic 1: 市场 指数 板块 行业 反弹 机会 结构调整 震荡 下跌 持续 资金 央企 新低 大盘 估值 券商 加息 行情 走势 空间 策略 股指 沪指 周期 回落 标的 低开 弱势 调整 兴业证券
Topic 2: 基金 独角兽 数码 销售 东蓝 飞利信 投资者 上市公司 千山药机 梅安森 项目 记者 销量 调研 业绩 收入 规模 财务数据 产品 调查 基金公司 净利润 募集 披露 战略 渠道 人士 年报 确认 配售
Topic 3: 增长 同比 企业 优先股 反馈 意见 公共 支出 证监会 收入 全国 深交所 披露 协议 预算收入 预算 经营 生态 合格 约定 财务 尔康制药 风险 服务 净利润 审核 下降 补充 问询 同业竞争
Topic 4: 闪崩 质押 跌停 市场 信托 股票 股价 持续 资金 净流入 计划 控股股东 南都电源 人士 平仓 东方海洋 业绩 产品 分析 交易日 因素 同济堂 西藏发展 调整 股通 风险 减持 上市公司 比例 波导股份
Topic 5: 市场 改革 制度 住房 房地产 住宅 租赁 房价 论坛 金融 伟民 企业 城市 机制 部门 用地 陆家嘴 供应 资金 建设 国立 中国证监会 允许 组织 增加 我国 发展 需求 农村 各方
Topic 6: 行业 评级 昨日 康得新 机构 资金 买入 给予 板块 同比 增长 看好 宁德时代 共有 生产 暴风集团 暴风 产品 拜腾 净流入 中国 汽车 证券日报 业务 全球 净利润 投资 记者 大单 工业
Topic 7: 企业 产业 机器人 电子 集成电路 手机 设计 行业 预计 产能 投资 松山湖 技术 东莞 国内 有效 全球 领域 信息产业 出货量 扫地 同比 制造业 芯片 大湾 科沃斯 天然气 增长 半导体 消费
Topic 8: 加息 利率 净流出 净流入 央行 实际 美联储 交易日 资金 主力资金 市场 成交 股市 银行 金额 全文 造成 特大 市场利率 名义利率 财富 基金 东方财富网 中国 明天 因素 联邦 利息 短期市场 连续
Top

In [179]:
# 训练模型
k = 20 # 选择主题数
lda_20 = LatentDirichletAllocation(n_components=k, max_iter=50, learning_method='batch')
lda_20.fit(dw_countvector)
# 6709文档，单词数：84884，分20个主题 耗时：2-3m

NameError: name 'dw_countvector' is not defined

In [46]:
# Doc Topics 结果
# 通过fit_transform函数,可以得到文档主题分布,Doc_Topic distribution matrix
doctopic_matrix = lda_20.fit_transform(countstay) # numpy.ndarray
print(doctopic_matrix)
print(doctopic_matrix.shape)

[[  4.38596494e-04   4.38596493e-04   4.38596494e-04 ...,   4.38596495e-04
    4.38596499e-04   4.38596493e-04]
 [  1.19887748e-01   5.26315793e-04   5.26315793e-04 ...,   5.26315792e-04
    5.26315796e-04   5.26315805e-04]
 [  1.31898828e-02   1.45544567e-01   2.60345395e-01 ...,   9.38264218e-02
    3.43867768e-02   6.04594937e-05]
 ..., 
 [  1.92307693e-04   1.92307694e-04   1.92307694e-04 ...,   1.92307693e-04
    1.92307693e-04   1.92307693e-04]
 [  2.27272730e-04   9.95681818e-01   2.27272729e-04 ...,   2.27272729e-04
    2.27272732e-04   2.27272729e-04]
 [  1.65016503e-04   1.65016504e-04   1.65016506e-04 ...,   1.65016504e-04
    1.65016504e-04   1.65016506e-04]]
(108, 20)


In [47]:
doctopic = pd.DataFrame(doctopic_matrix)
doctopic.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439,0.991667,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439
1,0.119888,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.870639,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526
2,0.013190,0.145545,0.260345,0.000060,0.140077,0.070779,0.000060,0.000060,0.097295,0.000060,0.000060,0.000060,0.000060,0.065828,0.078124,0.000060,0.000060,0.093826,0.034387,0.000060
3,0.000431,0.000431,0.000431,0.000431,0.000431,0.000431,0.000431,0.000431,0.000431,0.601352,0.000431,0.000431,0.000431,0.000431,0.000431,0.000431,0.000431,0.000431,0.390890,0.000431
4,0.000388,0.199665,0.000388,0.195999,0.000388,0.057001,0.000388,0.000388,0.085265,0.000388,0.000388,0.000388,0.000388,0.000388,0.164536,0.000388,0.000388,0.000388,0.292108,0.000388


In [48]:
global collist
collist = list(doctopic.columns)

def appendmax(sr):
    one = sr.idxmax()
    maxindex = pd.Series(one)
    sr = sr.append(maxindex)
    return sr

dfe = doctopic.apply(lambda x:appendmax(x), axis=1)
dfe.columns = collist + ['max']
dfe.to_csv('cluster.csv')
dfe.head()

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,max
0,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439,...,0.000439,0.991667,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439,0.000439,12.0
1,0.119888,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,...,0.000526,0.000526,0.870639,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,13.0
2,0.013190,0.145545,0.260345,0.000060,0.140077,0.070779,0.000060,0.000060,0.097295,0.000060,...,0.000060,0.000060,0.065828,0.078124,0.000060,0.000060,0.093826,0.034387,0.000060,2.0
3,0.000431,0.000431,0.000431,0.000431,0.000431,0.000431,0.000431,0.000431,0.000431,0.601352,...,0.000431,0.000431,0.000431,0.000431,0.000431,0.000431,0.000431,0.390890,0.000431,9.0
4,0.000388,0.199665,0.000388,0.195999,0.000388,0.057001,0.000388,0.000388,0.085265,0.000388,...,0.000388,0.000388,0.000388,0.164536,0.000388,0.000388,0.000388,0.292108,0.000388,18.0


In [49]:
label = pd.read_csv('labelled.csv', encoding='gb18030')
label.head()

,Unnamed: 0,id,statdate,text,预报内容,预警内容,预警级别,预警持续期,微博种类,length
0,0,1909/10/24,2017/8/22,http://t.cn/h91BSI【升级气象灾害应急响应通知】据市气象台监测，广州市达到启...,NaN,台风,NaN,NaN,包含链接,136
1,1,1909/10/25,2017/8/22,转发微博,NaN,NaN,NaN,NaN,转发微博,4
2,2,1909/12/19,2017/8/23,http://t.cn/h91BSI【台风紧急警报】今天下午16时，今年第13号台风“天鸽”...,NaN,台风,NaN,NaN,包含链接,146
3,3,1909/12/20,2017/8/23,【七天天气预报—“天鸽”上岸，仍有风雨】#天气话你知# #13号台风天鸽# 强台风“天鸽”已...,NaN,台风,NaN,NaN,NaN,120
4,4,1909/12/21,2017/8/23,http://t.cn/h91BSI【台风蓝色预警信号】今年第13号台风“天鸽”，23日17...,NaN,台风,NaN,NaN,包含链接,138


In [196]:
frames = [label, dfe['max']]
result = pd.concat(frames, axis = 1)
result.head()

,id,statdate,length,预报内容,预警内容,预警级别,预警持续期,微博种类,text,max
0,1909/10/24,2017/8/22,136,NaN,台风,NaN,NaN,包含链接,http://t.cn/h91BSI【升级气象灾害应急响应通知】据市气象台监测，广州市达到启...,0.0
1,1909/10/25,2017/8/22,4,NaN,NaN,NaN,NaN,转发微博,转发微博,2.0
2,1909/12/19,2017/8/23,146,NaN,台风,NaN,NaN,包含链接,http://t.cn/h91BSI【台风紧急警报】今天下午16时，今年第13号台风“天鸽”...,2.0
3,1909/12/20,2017/8/23,120,NaN,台风,NaN,NaN,NaN,【七天天气预报—“天鸽”上岸，仍有风雨】#天气话你知# #13号台风天鸽# 强台风“天鸽”已...,2.0
4,1909/12/21,2017/8/23,138,NaN,台风,NaN,NaN,包含链接,http://t.cn/h91BSI【台风蓝色预警信号】今年第13号台风“天鸽”，23日17...,2.0


In [50]:
print_top_words_name(lda_20, word_names, 15)

# 打印每个主题下权重较高的词
Topic 0: 三次 中信国安 万隆 上半年 上限 中小 与会者 一段时间 三一重工 久远银海 一致性 中原特钢 互联 五大 主导
Topic 1: 一举 一些则 一审 上升 一次能源 下限 万股 下降通道 三类 一流 三年 中共 三钢闽光 中国海防 中国人寿
Topic 2: 一半 万亿元 一两个 上市公司 万兴科技 下半年 万亿 一大 一部分 三联虹普 一级 一倍 一直维持在 一波三折 一线
Topic 3: 一两个 一去不返 万人 一切正常 一只 一是央企 三代 一纸 一行 一事 万邦德 一倍 三天 上游 下降
Topic 4: 一抢而空 一去不返 一款 一是央企 下游 上市规则 一行 七项 一两个 两日 万部 中下旬 一模一样 一切正常 上证指数
Topic 5: 一如既往 下称 一方 一大步 一字板 一事 一个月 一些则 一汽 专访 两位 一举 一直维持在 一块 两国
Topic 6: 一批 一月份 一只 一蹴而就 一事 三地 一笔 一倍 下快 不仅如此 两万元 不远 下发 一举 丰富
Topic 7: 下滑 不良率 三分之一 一日 互认 专区 万众瞩目 一如既往 一小部分 一方 三年 争夺 一百倍 三季度 一致性
Topic 8: 一一对 一出 一个月 一天 一周 一步 一心堂 一小部分 万户 一场 一汽集团 一日 一道 万亿美元 一大块
Topic 9: 一季度末 万钧 一个月 一度 一类 一场 万个 两翼 不良 一切正常 一路下行 东兴证券 中超 一大批 中葡股份
Topic 10: 三五十个 上月 上行 不明 专题会议 一次性 一流 一百倍 不应 个股方面 一级 价格波动 两大 中小板 以太
Topic 11: 万亿 万方 东岸 低至 一季度末 万兴科技 事实上 伟达 一部分 入驻 业绩 上攻 中粮 交易价格 主体
Topic 12: 一季度 临近 主题性 任性 中文在线 债务危机 一直维持在 一路下行 其三 下挫 个人所得税 二是 中国新闻网 一体 一两个
Topic 13: 一份 一季 一波 一条 一个月 一手 上证 一带一路 不变 万手 万至 上扬 两国关系 一枝独秀 万隆
Topic 14: 一切正常 上影 三分之一 不可逆转 一体化 为例 个股表现 一个月 一大步 上海 一次性 二手车 上日 万手 一举


In [51]:
def top_topics(model,topN):
    doctopic_matrix = model.fit_transform(countstay)
    doctopic = doctopic_matrix.argmax(axis=1).tolist() # 取最大值时的索引
    print("# 打印前 %d 热门主题的序号" % topN)
    print([Counter(doctopic).most_common(topN)[i][0] for i in range(topN)]) # 找出列表元素最多的N个,结果是list类型

In [52]:
top_topics(lda_50,10)
# 6709个文档，lda_50，前10个主题，耗时： 8 m 

# 打印前 10 热门主题的序号
[21, 15, 22, 36, 11, 38, 45, 23, 46, 44]


In [53]:
# 效果等同上述函数
doctopic_matrix = lda_20.fit_transform(countstay)
doctopic = doctopic_matrix.argmax(axis=1).tolist() # 取最大值时的索引
print("# 打印前 %d 热门主题的序号" % topN)
c = Counter(doctopic)
c.most_common(10) # 找出列表元素最多的N个,结果是list类型

NameError: name 'topN' is not defined

In [82]:
# 主题挖掘建模：文本聚类算法
# 特征处理: TF-IDF做文档特征
tfidf_vectorizer = TfidfVectorizer(stop_words=stop,max_df=0.9,min_df=10) # 特征提取
tfidf = tfidf_vectorizer.fit_transform(corpus) # matrix 
word_names = tfidf_vectorizer.get_feature_names() # list
# tfidf矩阵:元素a[i][j]表示j词在i文档中的tf-idf权重
# weight = tfidf.toarray()  # matrix -> ndarray
print(len(word_names))

1097


In [83]:
# 定义余弦相似度
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf)

In [84]:
# keans
# 训练模型
k = 5
km_5 = KMeans(n_clusters=k)
%time km_5.fit(tfidf)
clusters = km_5.labels_.tolist() # 每个文档所属类别

Wall time: 8.98 s


In [86]:
print("Top terms per cluster:")
print()
# 按离质心的距离排列聚类中心，由近到远
order_centroids = km_5.cluster_centers_.argsort()[:, ::-1] 

for i in range(k):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :15]: # 每个聚类选 15 个词
        print(' %s' % word_names[ind], end=' ')
    print() # 空行

Top terms per cluster:

Cluster 0: 天气话你知  广州  转发  微博  不语  天气  二哈  台风  简单  允悲  晚安  注意  冷空气  明天  早安 
Cluster 1: 广州市  气象台  多云  气温  各区  轻微  预测  发布  阴天  天气话你知  预计  晴到多云  偏北风  和缓  广州市今日天气 
Cluster 2: 随手拍广州  憧憬  不语  二哈  可爱  广州  简单  不错  摊手  天空  阳光  赏花  开心  太阳  花花 
Cluster 3: 预警  解除  我区  暴雨  气象台  信号  雷雨大风  黄色预警信号  黄色  黄埔区  发布  南沙  高温  花都区  生效 
Cluster 4: 小编  天气  不语  简单  二哈  允悲  广州  摊手  首席  回波  降水  泥萌  气温  出门  小伙伴 


In [87]:
km_5.labels_ # numpy.ndarray

array([0, 0, 0, ..., 0, 4, 4])

In [88]:
km_5.labels_.tolist()

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 4,
 3,
 3,
 3,
 0,
 0,
 0,
 3,
 0,
 1,
 3,
 3,
 0,
 0,
 4,
 4,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 1,
 0,
 0,
 3,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 3,
 4,
 3,
 3,
 0,
 0,
 2,
 0,
 0,
 2,
 3,
 4,
 2,
 1,
 2,
 4,
 0,
 0,
 0,
 4,
 4,
 1,
 4,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 4,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 4,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 3,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 3,
 3,
 0,
 3,
 0,
 0,
 3,
 3,
 0,
 0,
 3,
 0,
 3,
 3,
 0,
 3,
 4,
 4,
 0,
 4,
 3,
 3,
 4,
 4,
 0,
 0,
 4,
 1,
 0,
 0,
 3,
 2,
 3,
 0,
 3,
 3,
 0,
 0,
 4,
 3,
 1,
 0,
 0,
 4,
 4,
 0,
 1,
 0,
 2,
 0,
 2,
 1,
 0,
 2,
 1,
 4,
 4,
 0,
 4,
 4,
 1,
 3,
 4,
 0,
 0,
 1,
 0,
 0,
 2,
 1,
 1,
 3,
 3,
 3,
 0,
 0,
 0,
 3,
 0,
 0,
 0,


In [89]:
label = pd.Series(km_5.labels_.tolist())
label

0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      2
22      2
23      4
24      3
25      3
26      3
27      0
28      0
29      0
       ..
7398    4
7399    0
7400    4
7401    1
7402    4
7403    0
7404    0
7405    4
7406    2
7407    0
7408    4
7409    0
7410    0
7411    0
7412    0
7413    0
7414    0
7415    0
7416    1
7417    0
7418    4
7419    0
7420    0
7421    0
7422    4
7423    1
7424    1
7425    0
7426    4
7427    4
Length: 7428, dtype: int64

In [91]:
df = pd.read_csv('data/weibo_text_data.csv', encoding='gb18030')
df.head()

,id,statdate,text
0,1909/10/24,2017/8/22,http://t.cn/h91BSI【升级气象灾害应急响应通知】据市气象台监测，广州市达到启...
1,1909/10/25,2017/8/22,转发微博
2,1909/12/19,2017/8/23,http://t.cn/h91BSI【台风紧急警报】今天下午16时，今年第13号台风“天鸽”...
3,1909/12/20,2017/8/23,【七天天气预报—“天鸽”上岸，仍有风雨】#天气话你知# #13号台风天鸽# 强台风“天鸽”已...
4,1909/12/21,2017/8/23,http://t.cn/h91BSI【台风蓝色预警信号】今年第13号台风“天鸽”，23日17...


In [92]:
y = label.to_frame(name='所属类别')
y.head()
y.shape

(7428, 1)

In [93]:
result = pd.concat([df, y], axis=1)
result

,id,statdate,text,所属类别
0,1909/10/24,2017/8/22,http://t.cn/h91BSI【升级气象灾害应急响应通知】据市气象台监测，广州市达到启...,0
1,1909/10/25,2017/8/22,转发微博,0
2,1909/12/19,2017/8/23,http://t.cn/h91BSI【台风紧急警报】今天下午16时，今年第13号台风“天鸽”...,0
3,1909/12/20,2017/8/23,【七天天气预报—“天鸽”上岸，仍有风雨】#天气话你知# #13号台风天鸽# 强台风“天鸽”已...,0
4,1909/12/21,2017/8/23,http://t.cn/h91BSI【台风蓝色预警信号】今年第13号台风“天鸽”，23日17...,0
5,1909/12/22,2017/8/23,"#13号台风天鸽# 地铁4号线地面高架段恢复运营服务 ,",0
6,1909/12/23,2017/8/23,转发微博,0
7,1909/12/24,2017/8/23,转发微博,0
8,1909/12/25,2017/8/23,转发微博,0
9,1909/12/26,2017/8/23,转发微博,0


In [95]:
result.to_csv('labelled_KMeans_clustered.csv', encoding='gb18030')

In [98]:
exercise = pd.read_csv('labelled.csv', encoding='gb18030')
exercise

,id,statdate,length,预报内容,预警内容,预警级别,预警持续期,微博种类,所属类别,text
0,1909/10/24,2017/8/22,136,NaN,台风,NaN,NaN,包含链接,0.0,http://t.cn/h91BSI【升级气象灾害应急响应通知】据市气象台监测，广州市达到启...
1,1909/10/25,2017/8/22,4,NaN,NaN,NaN,NaN,转发微博,0.0,转发微博
2,1909/12/19,2017/8/23,146,NaN,台风,NaN,NaN,包含链接,0.0,http://t.cn/h91BSI【台风紧急警报】今天下午16时，今年第13号台风“天鸽”...
3,1909/12/20,2017/8/23,120,NaN,台风,NaN,NaN,NaN,0.0,【七天天气预报—“天鸽”上岸，仍有风雨】#天气话你知# #13号台风天鸽# 强台风“天鸽”已...
4,1909/12/21,2017/8/23,138,NaN,台风,NaN,NaN,包含链接,0.0,http://t.cn/h91BSI【台风蓝色预警信号】今年第13号台风“天鸽”，23日17...
5,1909/12/22,2017/8/23,29,NaN,台风,NaN,NaN,NaN,0.0,"#13号台风天鸽# 地铁4号线地面高架段恢复运营服务 ,"
6,1909/12/23,2017/8/23,4,NaN,NaN,NaN,NaN,转发微博,0.0,转发微博
7,1909/12/24,2017/8/23,4,NaN,NaN,NaN,NaN,转发微博,0.0,转发微博
8,1909/12/25,2017/8/23,4,NaN,NaN,NaN,NaN,转发微博,0.0,转发微博
9,1909/12/26,2017/8/23,4,NaN,NaN,NaN,NaN,转发微博,0.0,转发微博


In [100]:
exercise.loc[exercise.text.str.contains(0.0), '所属类别'] = '出行提示'
exercise.loc[exercise.text.str.contains(1.0), '所属类别'] = '天气预报'
exercise.loc[exercise.text.str.contains(2.0), '所属类别'] = '拍照发图'
exercise.loc[exercise.text.str.contains(3.0), '所属类别'] = '极端预警'
exercise.loc[exercise.text.str.contains(4.0), '所属类别'] = '微博表情'
exercise

TypeError: first argument must be string or compiled pattern

In [26]:
[Counter(km_20.labels_.tolist()).most_common(20)[i][0] for i in range(20)]

[0, 11, 19, 8, 7, 18, 13, 2, 1, 12, 3, 6, 5, 9, 14, 15, 17, 4, 10, 16]

In [27]:
Counter(km_20.labels_.tolist()).most_common(20)

[(0, 1605),
 (11, 1004),
 (19, 558),
 (8, 458),
 (7, 423),
 (18, 350),
 (13, 330),
 (2, 304),
 (1, 299),
 (12, 251),
 (3, 244),
 (6, 233),
 (5, 214),
 (9, 212),
 (14, 211),
 (15, 181),
 (17, 156),
 (4, 141),
 (10, 128),
 (16, 126)]